# Process the Customers Data
1. Ingest the data into the ata lakehouse - bronze_customers
2. Perform data quality checks and transform the data as requied - silver_customers_clean
3. Apply changes to the customers data - silver_customers

1. Ingest the data into the data lakehouse - bronze_customers

In [0]:
-- This operation requires unity catalog

CREATE OR REFRESH STREAMING TABLE bronze_customers
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT
  *,
  _metadata.file_path AS input_file_path,
  CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volume/circuitbox/landing/operational_data/customers',
  'json',
  map('cloudFiles.inferColumntypes', 'true')
);

2. Perform data quality checks and transform the data as required - silver_customers_clean

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customers_clean(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_customer_name EXPECT (customer_name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_telephone EXPECT (LENGTH(telephone) >= 10),
  CONSTRAINT valid_email EXPECT (email IS NOT NULL),
  CONSTRAINT valid_date_of_birth EXPECT (date_of_birth >= '1920-01-01')
)
COMMENT 'cleaned customer data'
TBLPROPERTIES('quality', 'true')
AS
SELECT
  customer_id,
  customer_name,
  CAST(date_of_birth AS DATE) AS date_of_birth,
  telephone,
  email,
  CAST(created_date AS DATE) AS created_date
FROM STREAM(LIVE.bronze_customers)

3. Apply changes to the Customers Data - silver_customers

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customers
COMMENT 'SCD Type 1 customers data'
TBLPROPERTIES("quality" = "true")

In [0]:
APPLY CHANGES INTO LIVE.silver_customers
FROM STREAM(LIVE.silver_customers_clean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1; -- Optional as type 1 is the default option